In [1]:
DAY = "24"

In [2]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"

INPUT_FILE = f"{DAY}.txt"

In [3]:
# Manual INPUT
INPUT ="""x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02
"""

In [4]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

## Usefull import

In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

In [6]:
DIR = {(0,1):'>',(1,0):'v',(0,-1):'<',(-1,0):'^'}
DIRd = {v:k for k,v in DIR.items()}

## Solution

In [7]:
OP ={
    'OR':(lambda x1,x2: x1 or x2),
    'XOR':(lambda x1,x2: x1 != x2),
    'AND':(lambda x1,x2: x1 and x2)
}

In [8]:
def sort_circuit(circuit,init_vals):
    sorted_circuit = []
    vals= set(init_vals)
    while len(sorted_circuit)<len(circuit):
        for c in circuit:
            if c in sorted_circuit:
                continue
            if c[0] in vals and c[2] in vals:
                sorted_circuit.append(c)
                vals.add(c[3])
    return sorted_circuit

In [9]:
val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
circuit = re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)
circuit = sort_circuit(circuit,val.keys())

In [10]:
for x1,oper,x2,res in circuit:
    val[res] = OP[oper](val[x1],val[x2])
z = 0
for var,value in val.items():
    if var[0]!='z':
        continue
    if value:
        digit = int(var[1:])
        z+= 1<<digit
z

58740594706150

In [11]:
val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
circuit = re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)
circuit_dep = {c[3]:c[:3] for c in circuit}


def tree(output,depth=-1):
    if output not in circuit_dep or depth==0:
        return output
    return f"{output}=({tree(circuit_dep[output][0],depth-1)} {circuit_dep[output][1]} {tree(circuit_dep[output][2],depth-1)})"

In [12]:
def new_part(p1,op,p2):
    """ Determine what part of the addition is the result of the operation.
        For example:
            (r,1) means the operation computes the reminder for the first digit
            (z,n) is the n-th digit of the addition result
            ...
    """
    x,dx = p1 if p1[0]<p2[0] else p2
    y,dy = p2 if p1[0]<p2[0] else p1
    if dx==0 and dy==0:
        if (x,y,op) == ('x','y','AND'):
            return ('r',1)
        if (x,y,op) == ('x','y','XOR'):
            return ('z',0)
    if (x,y,op)==('x','y','XOR') and dx==dy:
        return ('xoy',dx)
    if (x,y,op)==('x','y','AND') and dx==dy:
        return ('xay',dx)
    if (x,y,op)==('r','xoy','XOR') and dx==dy:
        return ('z',dx)
    if (x,y,op)==('r','xoy','AND') and dx==dy:
        return ('rxy',dx)
    if (x,y,op)==('rxy','xay','OR') and dx==dy:
        return ('r',dx+1) if dx+1<45 else ('z',dx+1)
    return None

In [13]:
def swapped_circuit(swap):
    circuit = {c[3]:list(c[:3]) for c in re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)}
    for o1,o2 in swap.items():
        circuit[o1],circuit[o2] = circuit[o2] ,circuit[o1]
    return circuit

def determine_all_parts(circuit):
    val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
    parts = {x:(x[0],int(x[1:])) for x in val.keys()}
    changed = True
    while changed:
        changed=False
        for res,(x,op,y) in circuit.items():
            if x in parts and y in parts and res not in parts:
                part = new_part(parts[x],op,parts[y])
                if part is not None:
                    parts[res]=part
                    changed=True
    return parts

In [14]:
## My solution involved 
##     - run the "determine_all_parts" without any swap
##     - manually find the last zXX that doesn't match a part ('z',xx) 
##     - Manually find the swap to make it work (using tree('zxx') for visualization)
##     - Add the swap and then redo the steps until everything works
swap= {'z14':'hbk'
       ,'kvn':'z18'
       ,'z23':'dbb'
       ,'cvh':'tfn'}

parts = determine_all_parts(swapped_circuit(swap))

In [15]:
a = list(swap.keys()) + list(swap.values())
a.sort()
','.join(a)

'cvh,dbb,hbk,kvn,tfn,z14,z18,z23'

## Automatic solution

In [18]:
def part_compose(part,digit):
    """
        Decompose a part of the equation into the operation to compute that part.
        For example:
           part_compose('z',12) => ('r',12),'XOR',('xoy',12) 
    """
    if part in ['x','y']:
        return None
    if part=='z':
        if digit==0:
            return ('x',0),'XOR',('y',0)
        if digit==45:
            return part_compose('r',digit)
        return ('r',digit),'XOR',('xoy',digit)
    if part == 'r':
        if digit==0:
            ('x',0),'AND',('y',0)
        return ('rxy',digit-1),'OR',('xay',digit-1)
    if part == 'rxy':
        return ('r',digit),'AND',('xoy',digit)
    if part == 'xay':
        return ('x',digit),'AND',('y',digit)
    if part == 'xoy':
        return ('x',digit),'XOR',('y',digit)
    return None

def reverse_find_error(circuit,parts,outs,current_signal,expected_part):
    """
        Find the error recursively.
            circuit = the circuit
            parts = dict giving what part of the equation is given in an output
            outs = dict giving what is the output of a part of the equation
        
            current_out = output that is incorrect
            current_part = expected part or the equation the output should be 
    """
    
    if expected_part in outs:
        # If the expected part of the equation is computed in another output, we simply return this ouput
        return current_signal,outs[expected_part]
    
    expected_part_1,expected_operation,expected_part_2 = part_compose(*expected_part)
    signal1,circuit_operation,signal2 = circuit[current_signal]
    
    if circuit_operation != expected_operation:
        raise Exception(f"Expecting '{expected_operation}', got '{circuit_operation}'")
        
    if signal1 not in parts and signal2 not in parts:
        raise Exception(f"Neither signals {signal1} or {signal2} are a part of the equation")
    
    def is_signal_correct(signal):
        return signal in parts and parts[signal] in [expected_part_1,expected_part_2]
    
    if is_signal_correct(signal1) and is_signal_correct(signal2):
        raise Exception("Both part are the correct equation, then equation is correct")
    
    if not is_signal_correct(signal1) and not is_signal_correct(signal2):
        raise Exception("Both part of the equation are incorrect, this algorithm cannot solve it")
    
    incorrect_signal = signal2 if is_signal_correct(signal1) else signal1
    correct_signal = signal1 if is_signal_correct(signal1) else signal2
    new_expected_part = expected_part_1 if parts[correct_signal]==expected_part_2 else expected_part_2
    
    return reverse_find_error(circuit,parts,outs,incorrect_signal,new_expected_part)
    

In [19]:
## automatic solution
is_correct = False
swap = dict()
while True: 
    sw_circuit = swapped_circuit(swap)
    parts = determine_all_parts(sw_circuit)
    outs = {v:k for k,v in parts.items()}
    
    stop = True
    for digit in range(46):
        z = f'z{digit:02}'
        z_part = ('z',digit)
        if z not in parts or parts[z]!=('z',digit):
            # z is incorrect
            stop = False
            break
    if stop:
        # no error found, all output are correct
        break
        
    # find the error for the z digit output
    out1,out2 = reverse_find_error(sw_circuit,parts,outs,z,z_part)
    swap[out1]=out2
    
a = list(swap.keys()) + list(swap.values())
a.sort()
','.join(a)

'cvh,dbb,hbk,kvn,tfn,z14,z18,z23'